# DS3 Kaggle Workshop - Advanced Practices in Pandas

Welcome to our Advanced Practices in Pandas Jupyter Notebook. This is from our Advanced Practices in Pandas Workshop on November 24, 2020 from 7-8 PM PST. We'd love to see you at our workshop!

With our interactive problems, we hope to guide you in your learning process. Here, you can practice useful pandas functions for DataFrame manipulation and analysis. Have fun!

The dataset we will be using is called [“Uber and Lyft Cab Prices”](https://www.kaggle.com/ravi72munde/uber-lyft-cab-prices?select=weather.csv) from Kaggle. For your convenience, we have downloaded it into the same repository as this Jupyter Notebook for you.

**Note:** The slideshow presentation will be published after the workshop. This will allow you to look back at the material covered and go over concepts that we were not able to get to during the timeframe.

## Importing Libraries and the Dataset

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
cab_rides = pd.read_csv('cab_rides.csv')
cab_rides

In [ ]:
weather = pd.read_csv('weather.csv')
weather

In [ ]:
weather.shape

## Concatenating Dataframes
[`pd.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) method: appends, or concatenates, two or more dataframes.
* Can be concatenated vertically (one atop another), which is default (axis = 0)
* Can also be concatenated horizontally (side-by-side) (axis = 1)

The argument for `pd.concat()` is an Iterable with elements of type DataFrame. 

Does it make more sense to concatenate vertically or horizontally with these datsets?
* Vertically (axis = 0)
* Horizontally (axis = 1)

In [ ]:
# axis = 1 for concat horizontally
pd.concat([cab_rides, weather], axis = 1)

We need to do a little bit of work if we want to be able to use `pd.concat()` . Simply putting one table atop another, or side-by-side does not help us analyze our data. More specifically, if we want to concatenate the tables horizontally, simply doing the code in the previous cell will not work because the locations in the `cab_rides` rows and `weather` rows will not correspond to one another. There will be a mismatch!

In order to concatenate the datasets, there needs to be commonality in the corresponding rows of the two dataframes. Moreover, the number of rows should be equal, otherwise you'll have a lot of NaN values after concatenating. Which column does it make sense to append from the two datasets?

In [ ]:
cab_rides.columns

In [ ]:
weather.columns

In [ ]:
cab_rides.time_stamp

In [ ]:
weather.time_stamp

It makes sense to consolidate these two datasets on the basis of 'time_stamp' first because weather changes with time, and it would be nice to have the cab rides with the weather at the particular time. However, we encounter an issue. The 'time_stamp' values in both datasets are very long (and seemingly arbitrary) integers. However, after reading the notes from the creator of this dataset, we can learn that these times are expressed in Unix Epoch time. The `weather` times are in seconds, but the `cab_rides` times are in milliseconds, which is why they look different as well.


Let's write a quick function `convert_unix_epoch_to_EST` to reformat Unix Epoch time to EST (Boston's time zone)! We will use `pd.apply()` to convert the times across all rows in both dataframes but we'll go deeper into `pd.apply()` later in the workshop.

In [ ]:
def convert_unix_epoch_to_EST(epoch_time_sec):
    epoch_time_sec = epoch_time_sec - 5 * 60 * 60   # subtract 3 hours because EST is GMT/UTC -5
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(epoch_time_sec))

In [ ]:
cab_rides['time_stamp'] = cab_rides['time_stamp'] / 1000.0 # cab_rides timestamps: ms --> s
cab_rides['time_stamp'] = cab_rides['time_stamp'].apply(convert_unix_epoch_to_EST) # apply function along column
weather['time_stamp'] = weather['time_stamp'].apply(convert_unix_epoch_to_EST) # weather timestamps already in sec

In [ ]:
cab_rides.head(3)

In [ ]:
weather.head(3)

Now let's find some basic summary statistics to see how well the `time_stamp` column in each dataframe matches up. Ideally, the minimum, maximum, and length (and all the values in the middle) will be the same if we want a perfect match. However, data is rarely perfect as we know!

In [ ]:
cab_rides.get('time_stamp').min(), cab_rides.get('time_stamp').max(), len(cab_rides.get('time_stamp'))

In [ ]:
weather.get('time_stamp').min(), weather.get('time_stamp').max(), len(weather.get('time_stamp'))

We can foresee a problem if we choose to concatenate the dataframes. The `cab_rides` and `weather` timestamps will not match, and this will hinder our analysis. We can try concatenating and observe this. But first, we need to sort both dataframes by their timestamps, since they are not already sorted by that column.

In [ ]:
cab_rides = cab_rides.sort_values(by = 'time_stamp').reset_index(drop = True)
weather = weather.sort_values(by = 'time_stamp').reset_index(drop = True)

In [ ]:
cab_rides.head(3)

In [ ]:
weather.head(3)

In [ ]:
concat = pd.concat([cab_rides, weather], axis = 1)
concat.head(3)
concat

In [ ]:
concat[['time_stamp']].head()

Problem that we predicted earlier: the timestamps for cab_rides and weather do not match, and the locations do not match either! Let's try another approach since `pd.concat()` does not seem to be effective.

## Merging Dataframes
[`pd.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) method: for combining 2 dataframes based on common data or indices
* Similar functionality to the SQL joins
* Much more flexible than `pd.concat()` or `pd.join()`
* Mandatory parameters: `left` and `right` dataframes but tons of optional parameters that help you merge on specific criteria
* Any matched columns with the same name will be listed only once in the merged dataframe to avoid repetition
* Any matched columns with different names will both be present in the merged dataframe, but will still be merged 
   * You can drop redundant columns if you'd like

We can try merging using the `on` parameter, which will merge both dataframes on the same column(s) passed as a list. Here's the catch: Both dataframes **must** share a column with that same name, which is often not the case.

After merging on 'time_stamp', does anyone notice any issues?

In [ ]:
pd.merge(cab_rides, weather, on = ['time_stamp']).head(3)

Since it's best to join on location and the associated time for the two dataframes, we can use two more explicit, and often more useful, parameters: `left_on` and `right_on`. 

Since `cab_rides` has 2 location columns, one for where the ride started ('source') and one for where it ended ('destination'), we need to make a decision. We can choose to have the 'source' from `cab_rides` match the 'location' from `weather` because people often have to wait outside for their rides, so this may be more impactful than the weather at their destination.

In [ ]:
rideshare_and_weather = pd.merge(cab_rides, weather, left_on = ['time_stamp', 'source'], 
                                 right_on = ['time_stamp', 'location']).head()
rideshare_and_weather.head()

A mini lesson on Joins (more examples on joins on [this](https://medium.com/swlh/merging-dataframes-with-pandas-pd-merge-7764c7e2d46d) Medium article by Ravjot Singh):
* The joins for `pd.merge()` have a very similar functionality to SQL joins.
* Inner join (default for `pd.merge()`): merges on only the rows that match within the `on` column(s) for both dataframes. This leads to the loss of most data because you're only keeping what is matching.
* Left join: will keep all the rows from the `left` dataframe even if it does not have a match from the right dataframe. Think about it like Inner (all matches) + all the remaining rows from the `left` dataframe. Rows from the `left` dataframe that do not match with a row from the `right` dataframe will have a NaN value for the column from the `right` dataframe.
* Right join: the mirror image of left join! Inner (all matches) + all the remaining rows from the `right` dataframe.
* Outer join: union of all rows from both dataframes, despite whether they match or not. This preserves most of the data, but leaves you with tons of NaN values.

There are definitely tradeoffs between the join, so choose carefully which will benefit you! If you do not want to use inner join, use the `how` parameter and specify the type of join.

Another useful parameter for `pd.merge()` is the `indicator` parameter. It will give a column called '_merge' and tell you what type of join that row was generated by.

In [ ]:
pd.merge(cab_rides, weather, left_on = ['time_stamp', 'source'], 
         right_on = ['time_stamp', 'location'], how = 'outer', indicator = True).head()

Merging on index: If the dataframes you're merging match on their indices, you can use `pd.merge()` for that too! Just use the `left_index` and `right_index` Boolean parameters. 

**NOTE**: You can use any combination: one of [`left_index`, `left_on`] and one of [`right_index`, `right_on`] if you happen to be matching the index of one dataframe with a column of the other dataframe.

In [ ]:
pd.merge(cab_rides, weather, left_index = True, right_index = True)

In general, `pd.concat()` is preferable over `pd.merge()` whenever you want to consolidate more than two dataframes. You cannot do this in `pd.merge()` unless you merge two dataframes and then merge a third dataframe to the merged dataframe, which often gets complicated. However, when using `pd.concat()`, the data has to perfectly match up, which is not very likely in real-world data. Mismatched data could be detrimental to our analysis and we wouldn't even know it if our dataset is huge! Although `pd.concat()` does have SQL join functionality, using `pd.merge()` requires less preparation on your end before the consolidation.

There is another way to consolidate two dataframes that is good to know. This is [`pd.join()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html), which is used to join 2 dataframes on their indices. This is a more rigid function, so `pd.merge()` is used more. We have also learned that we can use `pd.merge()` for joining on indices.

For more information on ways to consolidate dataframes, you can check out this [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) page!

## Transpose
[`pd.transpose()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transpose.html) method: reflects the dataframe over its main diagonal by writing rows as columns and vice-versa

"Transpose" means the process of exchanging places. This makes sense because we are exchanging places of the rows and columns (ie. interchanging the axes).



In [ ]:
cab_rides.shape # almost 70,000 rows and 10 columns !

What if you were to call transpose on the whole dataset?

In [ ]:
# commented so it doesn't take forever to run. 
# feel free to uncomment if you want to try the line!

# cab_rides.transpose()

... feels like it's running forever right? I would suggest taking this as a sign of NOT to call transpose on large datasets

So let's work with a smaller dataset and call transpose on that.

In [ ]:
five_cabs= cab_rides.head()
five_cabs

In [ ]:
five_cabs.shape

In [ ]:
five_cabs_transpose = five_cabs.transpose() #same as .T
five_cabs_transpose

In [ ]:
five_cabs_transpose.shape # see how the rows and columns switched?!

How about the weather dataset? Seems much smaller than cab_rides. Let's call transpose on that.

In [ ]:
weather.shape

In [ ]:
weather_transpose = weather.T # same as .transpose()
weather_transpose

What happens when you call transpose on a series instead of a dataframe?

In [ ]:
five_cab_names = five_cabs['name']
five_cab_names

In [ ]:
five_cab_names.transpose() # looks the same to me!

Why does a series behave this way? 

Series don't have the (row, column) format that dataframes do, so it just returns the same object. See below -- the column entry is empty!

In [ ]:
five_cab_names.shape

## Practice Problem: Transpose
Let's go back to the dataset, rideshare_and_weather, which you created in the last segment where you learned merge. You created this dataset by taking the first five entries of the cab_rides and weather merge. Practice transpose on that!

In [ ]:
rideshare_and_weather.shape

In [ ]:
rideshare_and_weather.transpose()

## Groupby


[`pd.groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) method: splits the object, applies a function, and combines the results (also referred to as split-apply-combine strategy)

`pd.groupby()` is a useful method often used in data science to group large amounts of data and compute operations on them.

Let's call `pd.groupby()` on `cab_rides` and pass in the parameter `cab_type` and see what it does.

In [ ]:
cab_rides.groupby('cab_type')

Calling `pd.groupby()` on the dataframe doesn't perform any operations; it just returns a DataFrameGroupBy object. In order to make use of `pd.groupby()`, you need to add some type of aggregate function to it.

Some aggregate functions you can use are: sum, mean, min, max.

Let's call the `sum` function on it.

In [ ]:
cab_rides.groupby('cab_type').sum()

cab_rides.groupby('cab_type') tells us that we are grouping the data in the cab_rides dataset by the cab_type. You can see that the indices on the newly formed table are in fact different cab types. By calling .sum(), it takes the sum of each column based on cab_type.

Why aren't all columns returned? Some columns don't make sense to be aggregated. For example, the column 'destination' was taken out because it didn't make sense to take the sum of 'destination.'

So let's take it one more step. How would you find the total distance (miles) done based on cab_type with the small dataframe you have above?


In [ ]:
# total distance miles done based on cab_type
sum_distance = cab_rides.groupby('cab_type')[['distance']].sum()
sum_distance

How about the total fare based on cab type?

In [ ]:
# total money spent on cab_type
sum_price = cab_rides.groupby('cab_type')[['price']].sum()
sum_price

## Practice Problem: Groupby
Practicing groupby on the weather dataset, how would you get the averages (mean) for each location?

In [ ]:
weather.head()

In [ ]:
# averages based on location
weather.groupby('location').mean()

How would you get just the average (mean) temperature based on location?

In [ ]:
# average temperature based on location
weather.groupby('location')[['temp']].mean()

You can elevate your `pd.groupby()` by adding on an apply function. This allows you to define and run your own custom function on the dataset, as opposed to using one of Pandas's pre defined aggregate functions. We will touch upon this later in the workshop.

## Apply
Let's get into the details of `pd.apply()` which enables us to apply a function along an axis of a DataFrame.

* Objects passed to the function are Series objects whose index is either the DataFrame’s index (axis=0) or the DataFrame’s columns (axis=1).

In [ ]:
def toft(df):
    ft = df * 5280
    return ft

In [ ]:
miles = cab_rides[['distance']]
feet = miles.apply(toft)

#reassign to original dataframe 
cab_rides = cab_rides.assign(distance_ft = feet)
cab_rides.head()

In [ ]:
cab_rides['distance_ft2'] = cab_rides['distance'].apply(toft)
cab_rides.head()

In [ ]:
cab_rides = cab_rides.drop(columns = 'distance_ft2')

## Lambda
 `pd.apply(lambda x:())` functions are defined using the keyword lambda. The function has only one expression and any number of arguements. A lambda function cannot contain any statements, and it returns a function object which can be assigned to any variable.

*  Allows for application of a custom function to satisy a goal during data manipulation such as developing a unique DataFrame column.
*  Useful for small tasks with less code. 

In [ ]:
def timeofday(x):
     if  x < 12:
        return 'Morning'
     elif 12<= x <18:
        return "Afternoon"
     else:
        return "Evening"

In [ ]:
type(cab_rides['time_stamp'][0])

In [ ]:
cabtime= pd.to_datetime(cab_rides['time_stamp'])
dft = pd.DataFrame(data=cabtime)
dft #new DataFrame time in datetime format

In [ ]:
cab_rides['time_stamp'] = dft['time_stamp']

In [ ]:
a = time.time()
cab_rides['time_of_day'] = cab_rides.time_stamp.dt.hour.apply(lambda row: timeofday(row))
b= time.time()
b-a

In [ ]:
cab_rides.head()

### Timing
When analyzing data, always have a plan of action. A difficult route could cost you a lot of time. Let's look at two ways of simplifying a variable for ease of use and further application.  

In [ ]:
def tod(time):
    t = time.str[11:13]
    tint = [int(x) for x in t]
    for i, x in enumerate(tint):
        if  x < 12:
             tint[i] = 'Morning'
        elif 12<= x <18:
            tint[i] = "Afternoon"
        else:
            tint[i] = "Evening"
    return tint

weather.time_stamp.str[11:13]

In [ ]:
a = time.time()
weather['Timeofday'] = tod(weather['time_stamp'])
b= time.time()
b-a, weather.head()

In [ ]:
weather['time_stamp'] = pd.to_datetime(weather['time_stamp'])
a = time.time()
weather['timeofday'] = weather['time_stamp'].dt.hour.apply(lambda row : timeofday(row))
b = time.time()
b-a, weather.head()

In [ ]:
weather.head()

While the calculation times of the two methods were quite similar, the actual time to code was much longer (double in this case !!!). Make sure to understand the particular data type you are working with and various functions available to you.

### When to Rideshare
Obviously sometimes you have to Uber and cannot wait for the time to be just right. Despite this fact, let's attempt to find the optimal weather to Uber or Lyft in terms of surge using the  `rideshare_and_weather` DataFrame. 

In [ ]:
rideshare_and_weather = pd.merge(cab_rides, weather, left_on = ['time_stamp', 'source'], right_on = ['time_stamp', 'location'])
rideshare_and_weather.head()

In [ ]:
def surge_type(x):
    if x > 1.0:
        return 'SURGE'
    else:
        return 'Normal'
        

In [ ]:
rideshare_and_weather['surge_type']= rideshare_and_weather.apply(lambda x: surge_type(x['surge_multiplier']),axis=1)

In [ ]:
surge = rideshare_and_weather[rideshare_and_weather['surge_type'] == 'SURGE'].sort_values(by = 'time_stamp')\
.reset_index(drop = True)
normal = rideshare_and_weather[rideshare_and_weather['surge_type'] == 'Normal'].sort_values(by = 'time_stamp')\
.reset_index(drop = True)

Take a look at the surge and normal rows to determine if their is an obvious discrepency between the two at the unique time of day. 

In [ ]:
S = surge.groupby('timeofday').mean()
S

In [ ]:
N = normal.groupby('timeofday').mean()
N

In [ ]:
fig, ax = plt.subplots()

a_heights, a_bins = np.histogram(surge['temp'])
b_heights, b_bins = np.histogram(normal['temp'].sample(n=150, random_state=1), bins=a_bins)

width = (a_bins[1] - a_bins[0])/3

ax.bar(a_bins[:-1], a_heights, width=width, facecolor='cornflowerblue')
ax.bar(b_bins[:-1]+width, b_heights, width=width, facecolor='seagreen')
#seaborn.despine(ax=ax, offset=10)
plt.xlabel('Temperature')
plt.ylabel('Count')
fig.legend(['surge','normal'],bbox_to_anchor =(0.75, 1), ncol = 2)

In [ ]:
fig, ax = plt.subplots()

a_heights, a_bins = np.histogram(surge['distance'])
b_heights, b_bins = np.histogram(normal['distance'].sample(n=150, random_state=1), bins=a_bins)

width = (a_bins[1] - a_bins[0])/3

ax.bar(a_bins[:-1], a_heights, width=width, facecolor='cornflowerblue')
ax.bar(b_bins[:-1]+width, b_heights, width=width, facecolor='seagreen')
#seaborn.despine(ax=ax, offset=10)
plt.xlabel('Distance (miles)')
plt.ylabel('Count') 
fig.legend(['surge','normal'],bbox_to_anchor =(0.75, 1), ncol = 2)

In [ ]:
fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4

normal['timeofday'].sample(n=145, random_state=1).value_counts().plot(kind='bar', color='red', ax=ax, width=width, position=1)
surge['timeofday'].value_counts().plot(kind='bar', color='blue', ax=ax2, width=width, position=0)

ax.set_ylabel('Number')
ax.set_ylim([0,70]);
ax2.set_ylim([0,70]);
fig.legend(['normal','surge'],bbox_to_anchor =(0.75, 1.15), ncol = 2)

ax2.axis('off')

plt.show()

In order to determine under what circumstances the surge_multiplier is applied, one would need more data. Plain and simple. Whether surge is related to weather or the time of day , at this point, it is uncertain. All we can do is assume and extrapolate our assumption. However, this is dangerous as human bias is a "guesstimate." 

*On your own* : Try a hypothesis test 


Possible ideas: 
- How does it seem like the number of rides are affected by the type of weather?
- Is there a relation between the surge multiplier of the ride and the weather quality?
- Does the length of the rides increase due to the poor weather quality?